In [1]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [2]:
# enforce reproducibility
torch.manual_seed(1)

In [3]:
dire = "./cleaned-data-processed" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 426, 1: 426}


In [4]:
# load pretrained model
vgg16 = torchvision.models.vgg16(pretrained=True)
googlenet = torchvision.models.googlenet(pretrained=True)
efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [5]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained, num_classes):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(self.network.fc.in_features, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, num_classes),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [6]:
# define model to replace here
model_details = {
    'model': resnet50,
    'name': 'resnet50'

#     'model': efficientnet_b7,
#     'name': 'efficientnet_b7'

#     'model': googlenet,
#     'name': 'googlenet'

#     'model': vgg16,
#     'name': 'vgg16' 
}

In [7]:
# define params here
num_classes = 2
model_name = model_details['name']
net = ClassifierNet(model_details['model'], num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.005)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 20

training.run(model_name, net, optimizer, scheduler, criterion, epochs, num_classes, trainloader, testloader)

Training on cuda


  0%|          | 0/20 [00:00<?, ?it/s]